<a href="https://colab.research.google.com/github/chaeyeon2367/ml-python-NYCtaxiDemandForecast/blob/main/Light_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Basic preprocessing for modeling
- Purpose
- Basic preprocessing for modeling
- the heart of one's mouth
- What is possible with SQL (BigQuery) is that SQL may perform better than => local Python. However, data IO must also be considered.
- Hard preprocessing in SQL is in Python! => Label Encoding, One Hot Encoding 등
- the contents of a story
 - `Pre-processing related to time`
     - Extract MONTH, HOUR, WEEKDAY, etc. from DATETIME
     - BigQuery vs Python
 - Reverse Geocoding to `BigQuery GIS`
     - Convert coordinates => zip_code
 - `Categorical data preprocessing`
     - One Hot Encoding
     - Label Encoding
 - `Train / Test Split`

In [6]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import warnings
import seaborn as sns
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')

In [4]:
from google.colab import auth
auth.authenticate_user()

In [7]:
#Replace 'project_id' with your BigQuery project ID
from google.cloud import bigquery
client = bigquery.Client(project='nyctaxi-demand-forecast')


### Time-related preprocessing
- 1) Handled by BigQuery
   - DATETIME_TRUNC : Data truncation
   - Extract MONTH, WEEK, DAY, HOUR, etc. easily using the EXTRACT function
      - [EXTRACT official document](https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions?hl=ko#extract)
   - The FORMAT_DATETIME function allows you to extract the days of the week (however, 1 is Monday => 0 is Monday in Pantas)
      - [FORMAT_DATETIME official document](https://cloud.google.com/bigquery/docs/reference/standard-sql/datetime_functions?hl=ko#format_datetime)

In [8]:
%%time
extract_query = """
SELECT
    *,
    EXTRACT(MONTH FROM pickup_hour) AS month,
    EXTRACT(DAY FROM pickup_hour) AS day,
    CAST(format_datetime('%u', pickup_hour) AS INT64) -1 AS weekday, #To match Python, -1
    EXTRACT(HOUR FROM pickup_hour) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_hour) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend
FROM (
    SELECT 
        DATETIME_TRUNC(pickup_datetime, hour) AS pickup_hour,
        count(*) AS cnt
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015` 
    WHERE EXTRACT(MONTH from pickup_datetime) = 1
    GROUP BY pickup_hour
)
ORDER BY pickup_hour
"""

df = client.query(extract_query).to_dataframe()

CPU times: user 84.1 ms, sys: 12.4 ms, total: 96.5 ms
Wall time: 3.98 s


In [9]:
df.tail()

,pickup_hour,cnt,month,day,weekday,hour,is_weekend
739,2015-01-31 19:00:00+00:00,32426,1,31,5,19,1
740,2015-01-31 20:00:00+00:00,27542,1,31,5,20,1
741,2015-01-31 21:00:00+00:00,27467,1,31,5,21,1
742,2015-01-31 22:00:00+00:00,29856,1,31,5,22,1
743,2015-01-31 23:00:00+00:00,29850,1,31,5,23,1


- Alternatively, we can query below.
    - However, we can currently query the following on an hour-by-hour basis, and if you divide it by 30 minutes, we need to adjust the minutes.
    - Example: 14:35 => 14:30 Like this.

In [11]:
%%time
extract_query = """
SELECT 
    DATETIME_TRUNC(pickup_datetime, hour) AS pickup_hour,
    EXTRACT(MONTH FROM pickup_datetime) AS month,
    EXTRACT(DAY FROM pickup_datetime) AS day,
    CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) -1 AS weekday, # Python과 맞추기 위해 -1
    EXTRACT(HOUR FROM pickup_datetime) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend,
    count(*) AS cnt
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015` 
WHERE EXTRACT(MONTH from pickup_datetime) = 1
GROUP BY pickup_hour, month, day, weekday, hour, is_weekend
ORDER BY pickup_hour
"""

df = client.query(extract_query).to_dataframe()

CPU times: user 83.3 ms, sys: 13.6 ms, total: 96.9 ms
Wall time: 4.78 s


In [10]:
df.tail()

,pickup_hour,cnt,month,day,weekday,hour,is_weekend
739,2015-01-31 19:00:00+00:00,32426,1,31,5,19,1
740,2015-01-31 20:00:00+00:00,27542,1,31,5,20,1
741,2015-01-31 21:00:00+00:00,27467,1,31,5,21,1
742,2015-01-31 22:00:00+00:00,29856,1,31,5,22,1
743,2015-01-31 23:00:00+00:00,29850,1,31,5,23,1


- 2) Processing in Python
    - Processing Pandas datetime
    - Convert datetime to `pd.to_datetime` and extract to df[col].dt.xxx

In [12]:
extract_query = """
SELECT 
    DATETIME_TRUNC(pickup_datetime, hour) AS pickup_hour,
    count(*) AS cnt
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015` 
WHERE EXTRACT(MONTH from pickup_datetime) = 1
GROUP BY pickup_hour
ORDER BY pickup_hour
"""

df = client.query(extract_query).to_dataframe()

In [13]:
df.head()

,pickup_hour,cnt
0,2015-01-01 00:00:00+00:00,28312
1,2015-01-01 01:00:00+00:00,31707
2,2015-01-01 02:00:00+00:00,28068
3,2015-01-01 03:00:00+00:00,24288
4,2015-01-01 04:00:00+00:00,17081


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   pickup_hour  744 non-null    datetime64[ns, UTC]
 1   cnt          744 non-null    Int64              
dtypes: Int64(1), datetime64[ns, UTC](1)
memory usage: 12.5 KB


- Pickup_hour is datetime64, so we can extract date-related data without having to_datetime.
   - Convert to `pd.to_datetime(df['pickup_hour')` if it is in a different form

In [15]:
df['month'] = df['pickup_hour'].dt.month
df['day'] = df['pickup_hour'].dt.day
df['weekday'] = df['pickup_hour'].dt.weekday
df['hour'] = df['pickup_hour'].dt.hour
df['is_weekend'] = (df['pickup_hour'].dt.weekday // 5 == 1).astype(int)

In [18]:
df.tail()

,pickup_hour,cnt,month,day,weekday,hour,is_weekend
739,2015-01-31 19:00:00+00:00,32426,1,31,5,19,1
740,2015-01-31 20:00:00+00:00,27542,1,31,5,20,1
741,2015-01-31 21:00:00+00:00,27467,1,31,5,21,1
742,2015-01-31 22:00:00+00:00,29856,1,31,5,22,1
743,2015-01-31 23:00:00+00:00,29850,1,31,5,23,1


- Summary: SQL (BigQuery) can perform better than => local Python in SQL. However, data IO must also be considered.


In [19]:
df.head(3)

,pickup_hour,cnt,month,day,weekday,hour,is_weekend
0,2015-01-01 00:00:00+00:00,28312,1,1,3,0,0
1,2015-01-01 01:00:00+00:00,31707,1,1,3,1,0
2,2015-01-01 02:00:00+00:00,28068,1,1,3,2,0


### Reverse Geocoding to `BigQuery GIS`Reverse Geocoding
- Convert coordinates => zip_code
- BigQuery `bigquery-public-data.geo_us_boundaries.zip_codes` 
- Extractable as ST_CONTAINS function

In [21]:
%%time
base_query = """
WITH base_data AS 
(
  SELECT nyc_taxi.*, gis.* EXCEPT (zip_code_geom)
  FROM (
    SELECT *
    FROM `bigquery-public-data.new_york.tlc_yellow_trips_2015`
    WHERE 
        EXTRACT(MONTH from pickup_datetime) = 1
        and pickup_latitude  <= 90 and pickup_latitude >= -90
    ) AS nyc_taxi
  JOIN (
    SELECT zip_code, state_code, state_name, city, county, zip_code_geom
    FROM `bigquery-public-data.geo_us_boundaries.zip_codes`
    WHERE state_code='NY'
    ) AS gis 
  ON ST_CONTAINS(zip_code_geom, st_geogpoint(pickup_longitude, pickup_latitude))
)

SELECT 
    zip_code,
    DATETIME_TRUNC(pickup_datetime, hour) as pickup_hour,
    EXTRACT(MONTH FROM pickup_datetime) AS month,
    EXTRACT(DAY FROM pickup_datetime) AS day,
    CAST(format_datetime('%u', pickup_datetime) AS INT64) -1 AS weekday,
    EXTRACT(HOUR FROM pickup_datetime) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend,
    COUNT(*) AS cnt
FROM base_data 
GROUP BY zip_code, pickup_hour, month, day, weekday, hour, is_weekend
ORDER BY pickup_hour


"""

base_df = client.query(base_query).to_dataframe()

CPU times: user 114 ms, sys: 41 ms, total: 155 ms
Wall time: 8.42 s


In [ ]:
base_df.tail()

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt
87015,10467,2015-01-31 23:00:00,1,31,5,23,1,1
87016,11375,2015-01-31 23:00:00,1,31,5,23,1,2
87017,11378,2015-01-31 23:00:00,1,31,5,23,1,1
87018,10459,2015-01-31 23:00:00,1,31,5,23,1,1
87019,11436,2015-01-31 23:00:00,1,31,5,23,1,1


### `Categorical Data preprocessing`
- One Hot Encoding

In [22]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(base_df[['zip_code']])

OneHotEncoder(handle_unknown='ignore')

In [23]:
ohe_output = enc.transform(base_df[['zip_code']]).toarray()
oh_feature_df = pd.concat([base_df, pd.DataFrame(ohe_output, columns='zip_code_'+enc.categories_[0])], axis=1)
oh_feature_df.head(3)

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,...,zip_code_12601,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801
0,10170,2015-01-01 00:00:00+00:00,1,1,3,0,0,44,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11231,2015-01-01 00:00:00+00:00,1,1,3,0,0,55,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10040,2015-01-01 00:00:00+00:00,1,1,3,0,0,13,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- Label Encoding

In [ ]:
# le = LabelEncoder()
# le.fit(base_df['zip_code'])
# base_df['zip_code_le'] = le.transform(base_df['zip_code'])

- We will use One Hot Encoding for Linear Regression

### Train and Test Split
- Time series data should not be random sampling
- Train should have historical data, Test should have future data
    - What if Train has future data? a prediction of the past by looking at the future
    - Predict future data based on historical data when used in practice

In [24]:
def split_train_and_test(df, date):
    """
  Dataframe divided by train_df and test_df
    df : Time series data frame
    date : Reference point date
    """
    train_df = df[df['pickup_hour'] < date]
    test_df = df[df['pickup_hour'] >= date]
    return train_df, test_df

- The above method may not like the way you explicitly specify a date
    - In this case, the time data in the Dataframe can only be designated as a Test for the last week.

In [25]:
def split_train_and_test_period(df, period):
  
    criteria = (max(df['pickup_hour']) - pd.Timedelta(days=period)).date()
    train_df = df[df['pickup_hour'] < criteria]
    test_df = df[df['pickup_hour'] >= criteria]
    return train_df, test_df

### Train / Test Split

In [37]:
train_df, test_df = split_train_and_test(oh_feature_df, '2015-01-24')

In [38]:
train_df.head()

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,...,zip_code_12601,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801
0,10170,2015-01-01 00:00:00+00:00,1,1,3,0,0,44,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11231,2015-01-01 00:00:00+00:00,1,1,3,0,0,55,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10040,2015-01-01 00:00:00+00:00,1,1,3,0,0,13,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10462,2015-01-01 00:00:00+00:00,1,1,3,0,0,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11225,2015-01-01 00:00:00+00:00,1,1,3,0,0,10,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
test_df.head()

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,...,zip_code_12601,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801
65118,11421,2015-01-24 00:00:00+00:00,1,24,5,0,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65119,11232,2015-01-24 00:00:00+00:00,1,24,5,0,1,8,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65120,11001,2015-01-24 00:00:00+00:00,1,24,5,0,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65121,11105,2015-01-24 00:00:00+00:00,1,24,5,0,1,14,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65122,11203,2015-01-24 00:00:00+00:00,1,24,5,0,1,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
del train_df['zip_code']
del train_df['pickup_hour']
del test_df['zip_code']
del test_df['pickup_hour']
# del train_df['zip_code_le']
# del test_df['zip_code_le']

In [41]:
train_df.head(2)

,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,zip_code_10003,zip_code_10004,...,zip_code_12601,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801
0,1,1,3,0,0,44,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,3,0,0,55,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
y_train = train_df.pop('cnt')
x_train = train_df.copy()

In [43]:
y_test = test_df.pop('cnt')
x_test = test_df.copy()

In [44]:
x_train

,month,day,weekday,hour,is_weekend,zip_code_10001,zip_code_10002,zip_code_10003,zip_code_10004,zip_code_10005,...,zip_code_12601,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801
0,1,1,3,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,3,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,3,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,3,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,3,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65113,1,23,4,23,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65114,1,23,4,23,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65115,1,23,4,23,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65116,1,23,4,23,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
y_train

0          44
1          55
2          13
3           3
4          10
         ... 
65113       2
65114     426
65115    1270
65116      18
65117      12
Name: cnt, Length: 65118, dtype: Int64

In [46]:
x_test

,month,day,weekday,hour,is_weekend,zip_code_10001,zip_code_10002,zip_code_10003,zip_code_10004,zip_code_10005,...,zip_code_12601,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801
65118,1,24,5,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65119,1,24,5,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65120,1,24,5,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65121,1,24,5,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65122,1,24,5,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87015,1,31,5,23,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87016,1,31,5,23,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87017,1,31,5,23,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87018,1,31,5,23,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y_test